In [1]:
#node sampler - sample more relevant (informative) nodes 
# qagenerator - generate questions to the sampled nodes
from evaluate_rag_utils import NodeSampler, QAGenerator, Validator

from llama_index.llms import OpenAI
from llama_index.finetuning import EmbeddingQAFinetuneDataset
import pandas as pd
import asyncio
import json

# make the async code working in the notebook cells
import nest_asyncio

nest_asyncio.apply()

from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator, FaithfulnessEvaluator
from llama_index import StorageContext, load_index_from_storage
from llama_index import ServiceContext
from openai import AsyncOpenAI



In [2]:
# load question: node dataset
qa_dataset_1000 = EmbeddingQAFinetuneDataset.from_json('question_node_dataset_1000.json')
# get the questions
eval_questions = qa_dataset_1000.queries.values()

# load indexed nodes
storage_cntxt_512 = StorageContext.from_defaults(persist_dir="../../data/index_storage_512")
idx_512 = load_index_from_storage(storage_cntxt_512)
storage_cntxt_1024 = StorageContext.from_defaults(persist_dir="../../data/index_storage_1024")
idx_1024 = load_index_from_storage(storage_cntxt_1024)

In [5]:
async def main():
    client = AsyncOpenAI()
    semaphore = asyncio.Semaphore(100)
    params_list = [
        {
            'client': client,
            'index': idx_1024,
            'semaphore': semaphore
            },
        {
            'client': client,
            'index': idx_512,
            'semaphore': semaphore,
            'has_node_postprocessors': True
            'num_nodes': 1
            }
    ]
    # classes that validate QA dataset with default evaluators
    validators = [Validator(**params) for params in params_list]

    output_paths = [
        'rag_evaluate_json_1024.json',
        'rag_evaluate_json_512_postprocess_both_1.json',
    ]

    # generate Q-A for 2 questions and validate them
    async with asyncio.TaskGroup() as tg:
                    tasks = [
                        tg.create_task(
                            validator.answers_evaluated_list(
                                queries=list(eval_questions)[:2],
                                output_path=output_path
                                )
                        )
                        for validator, output_path in zip(validators, output_paths)
                    ]

if __name__ == '__main__':
    asyncio.run(main())

len information = 3
len information = 14
len information = 3
len information = 7


In [3]:
# get more relevant nodes
# having higher counts of relevant and lower count of irrelevant words
# limit sample to 5% of all nodes
node_sampler = NodeSampler('relevant_words.txt', 'irrelevant_words.txt')
informative_nodes_1000 = node_sampler.informative_nodes(
    video_info_path='../../data/video_info.json', 
    chunk_size=1024, 
    fraction=0.05
    )

informative_nodes_3000 = node_sampler.informative_nodes(
    video_info_path='../../data/video_info.json', 
    chunk_size=1024*3, 
    fraction=0.05
    )


nodes_1000 = informative_nodes_1000[:10]
nodes_3000 = informative_nodes_3000[:10]

In [3]:
# initialize the model
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-1106")

# create QA generator
qa_generator = QAGenerator(llm=llm)

In [6]:
# generate:
# i) question-node pairs, save as json at qa_json_path
# ii) question-document pairs at doc_question_path
qa_dataset_1000 = qa_generator.generate_document_question_pairs(
    nodes=nodes_1000,
    doc_json_path='../../data/video_info.json',
    doc_question_path='doc_question_dataset_1000.json',
    qa_json_path='question_node_dataset_1000.json'
)

qa_dataset_3000 = qa_generator.generate_document_question_pairs(
    nodes=nodes_3000,
    doc_json_path='../../data/video_info.json',
    doc_question_path='doc_question_dataset_3000.json',
    qa_json_path='question_node_dataset_3000.json'
)

100%|██████████| 10/10 [00:18<00:00,  1.88s/it]


In [2]:
qa_dataset_1000 = EmbeddingQAFinetuneDataset.from_json('question_node_dataset_1000.json')

In [3]:
qa_dataset_1000.queries.values()

dict_values(['"Как работает алгоритм градиентного бустинга в анализе данных?"', '"Какие основные компетенции должен иметь аналитик данных?"', '"Что такое атрибуты и методы в Pandas DataFrame?"', '"Как дерево решений выбирает вопрос для прогнозирования данных?"', 'Какие навыки должен иметь аналитик данных согласно тексту?', '"Какие изменения были внесены в запрос для анализа данных?"', 'Какие хардскиллы важны для аналитика данных в сфере программирования?', 'Какие инструменты аналитики рекомендуются для работы с данными?', 'Как работает метод изоляционного леса для обнаружения выбросов?', '"Какие навыки должен иметь аналитик данных согласно тексту?"'])

In [6]:
storage_cntxt = StorageContext.from_defaults(persist_dir="../../data/index_storage_1024")
idx = load_index_from_storage(storage_cntxt)
print("Index is loaded")

eval_questions = qa_dataset_1000.queries.values()


# service_context_gpt3 = ServiceContext.from_defaults(llm=llm)
# rel_evaluator_gpt3 = RelevancyEvaluator(service_context=service_context_gpt3)
# faith_evaluator_gpt3 = FaithfulnessEvaluator(service_context=service_context_gpt3)

Index is loaded


In [9]:
client = AsyncOpenAI()

In [10]:
validator = Validator(
    client=client,
    index=idx,
)

In [8]:
query_str = (
    list(qa_dataset_1000.queries.values())[0]
)
query_engine = idx.as_query_engine()
response_vector = query_engine.query(query_str)



In [9]:
eval_result = rel_evaluator_gpt3.evaluate_response(
    query=query_str, response=response_vector
)


In [10]:
# define jupyter display function
def display_eval_df(query: str, response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [11]:
display_eval_df(query_str, response_vector, eval_result)

,Query,Response,Source,Evaluation Result
0,"""Как работает алгоритм градиентного бустинга в анализе данных?""","Алгоритм градиентного бустинга в анализе данных работает путем комбинирования множества деревьев решений для улучшения точности прогнозирования. Каждое новое дерево решений является самостоятельным алгоритмом машинного обучения, который использует те же данные, что и предыдущие деревья. Каждое дерево способно давать прогноз самостоятельно. Градиентный бустинг использует градиентный спуск для постепенного улучшения прогноза, минимизируя ошибку между прогнозами и фактическими значениями. Этот процесс повторяется до достижения определенного критерия остановки, такого как достижение определенного числа деревьев или улучшение прогноза не наблюдается.","Ребята, всем привет! Меня зовут Григорий Бударагин, я аналитик-разработчик компании Яндекс. Сегодня я расскажу вам механизм работы градиентного бустинга. Что же такое градиентный бустинг? Это алгоритм машинного обучения, который помогает прогнозировать различные числовые величины. Например, он используется в различных больших технологических компаниях для ранжирования веб-страниц, для выставления цен в приложении такси, а также для противодействия мошенничеству и многого другого. Не только градиентный бустинг, но и другие алгоритмы машинного обучения учатся прогнозировать числовые величины на каких-то исторических наблюдениях. Например, чтобы научить алгоритм предсказывать цену дома, ему необходимо наблюдение о проданных в прошлом домах с их ценами и другими измеряемыми характеристиками....",Pass


In [ ]:
async def get_llm_answer(
    query_engine,
    query: str,
    context_prompt: str = None,
    model_name: str = "gpt-3.5-turbo-1106",
) -> str:

    retrival = await query_engine.aquery(message)
    information = [
        (i.text, i.metadata["url"], i.metadata["title"]) for i in retrival.source_nodes
    ]

    for text in [text for text, _, _ in information]:
        logging.info(text)

    information_text = escape_html(" ".join([text for text, _, _ in information]))
    information_url = "\n".join(
        set(
            f'&#x25CF; <a href="{url}">{escape_html(title)}</a>'
            for _, url, title in information
        )
    )

    context_prompt = (
        "Ниже мы предоставили контекстную информацию\n"
        "---------------------\n"
        f"{information_text}"
        "\n---------------------\n"
        f"Учитывая эту информацию, ответьте, пожалуйста, на вопрос: {message}\n"
        "\n---------------------\n"
        "Ответ на вопрос должен быть развернутым, полным, и охватывать множество "
        "аспектов заданного вопроса"
        "Внимание! В ответе нельзя упоминать конекстную информацию! "
        "Пользователь не знает о ее наличии!"
    )

    logging.info("Сообщение сформировано и отправлено в OpenAI")
    model_name = "gpt-3.5-turbo-1106"

    context_response = await client.chat.completions.create(
        model=model_name, temperature=0, messages=[{"role": "user", "content": context_prompt}]
    )

In [12]:
a = 'ab{c}_{d}'
a

'ab{c}_{d}'

In [17]:
a.format_map({'c': 4, 'd': 1})

'ab4_1'